In [1]:
import pickle
import pandas as pd
import numpy as np
import os, sys, gc 
from plotnine import *
import plotnine

from tqdm import tqdm_notebook
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
from matplotlib import rc
import re
from matplotlib.ticker import PercentFormatter
import datetime
from math import log # IDF 계산을 위해

In [2]:
# 데이터 : https://www.kaggle.com/sengzhaotoo/movielens-small
# 원본 데이터 : https://grouplens.org/datasets/movielens/100k/

In [3]:
movie = pd.read_csv("./ratings.csv")

In [4]:
movie.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


## Implicit 을 이용한 ALS 구현

In [5]:
user2idx = {}
for i, l in enumerate(movie['userId'].unique()):
    user2idx[l] = i

movie2idx = {}
for i, l in enumerate(movie['movieId'].unique()):
    movie2idx[l] = i

In [6]:
idx2user = {i: user for user, i in user2idx.items()}
idx2movie = {i: item for item, i in movie2idx.items()}

In [7]:
useridx = movie['useridx'] = movie['userId'].apply(lambda x: user2idx[x]).values
movieidx = movie['movieidx'] = movie['movieId'].apply(lambda x: movie2idx[x]).values
rating = movie['rating'].values

In [8]:
import scipy  

purchases_sparse = scipy.sparse.csr_matrix((rating, (useridx, movieidx)), shape=(len(set(useridx)), len(set(movieidx))))

In [9]:
from implicit.evaluation import  *
from implicit.als import AlternatingLeastSquares as ALS
from implicit.bpr import BayesianPersonalizedRanking as BPR

In [10]:
als_model = ALS(factors=20, regularization=0.08, iterations = 20)
als_model.fit(purchases_sparse.T)

In [11]:
idx2rec = {}
for u in tqdm_notebook(idx2user.keys()):
    item_rec = als_model.recommend(u, purchases_sparse, N=100)
    item_rec = [idx2movie[x[0]] for x in item_rec]
    idx2rec[idx2user[u]] = item_rec

C:\Users\choco\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
